In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/code/data/train/subtask_1/en/')
#os.chdir('/content/drive/My Drive/Colab Notebooks/code/data/test/subtask_1/en/')

Mounted at /content/drive


In [ ]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
import re
import openai
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, classification_report, accuracy_score

In [ ]:
from google.colab import userdata
api_key = userdata.get('api-key')

In [ ]:
df_human = pd.read_csv("df_human.tsv", sep='\t')
df_bloom1b7 = pd.read_csv("df_bloom1b7.tsv", sep='\t')
df_bloom3b = pd.read_csv("df_bloom3b.tsv", sep='\t')
df_bloom7b1 = pd.read_csv("df_bloom7b1.tsv", sep='\t')
df_gpt = pd.read_csv("gpt.csv")

In [ ]:
df_samples = []
# 5 samples Bloom (classification outcome)
sample = df_bloom1b7[df_bloom1b7['id'] == 11106]
df_samples.append(sample)
sample = df_bloom1b7[df_bloom1b7['id'] == 798]
df_samples.append(sample)
sample = df_bloom1b7[df_bloom1b7['id'] == 729]
df_samples.append(sample)
sample = df_bloom7b1[df_bloom7b1['id'] == 25290]
df_samples.append(sample)
sample = df_bloom3b[df_bloom3b['id'] == 1022]
df_samples.append(sample)

# 5 samples no classification outcome (Bloom/human)
sample = df_bloom7b1[df_bloom7b1['id'] == 25349]
df_samples.append(sample)
sample = df_human[df_human['id'] == 18336]
df_samples.append(sample)
sample = df_human[df_human['id'] == 6804]
df_samples.append(sample)
sample = df_human[df_human['id'] == 26159]
df_samples.append(sample)
sample = df_bloom7b1[df_bloom7b1['id'] == 13024]
df_samples.append(sample)

# 5 samples human (classification outcome)
sample = df_human[df_human['id'] == 18934]
df_samples.append(sample)
sample = df_human[df_human['id'] == 6703]
df_samples.append(sample)
sample = df_human[df_human['id'] == 30729]
df_samples.append(sample)
sample = df_human[df_human['id'] == 3691]
df_samples.append(sample)
sample = df_human[df_human['id'] == 28685]
df_samples.append(sample)

# Added 5 random samples from GPT dataset
sample = df_gpt[df_gpt['id'] == 40000]
df_samples.append(sample)
sample = df_gpt[df_gpt['id'] == 40500]
df_samples.append(sample)
sample = df_gpt[df_gpt['id'] == 40770]
df_samples.append(sample)
sample = df_gpt[df_gpt['id'] == 40123]
df_samples.append(sample)
sample = df_gpt[df_gpt['id'] == 40990]
df_samples.append(sample)

df_samples = pd.concat(df_samples, ignore_index=True)

In [ ]:
df_samples

,id,prompt,text,label,model,domain
0,11106,Extremely happy I got to,Extremely happy I got to go down in the trench...,generated,A,tweets
1,798,(2) It follows from applying the rules and cri...,*/. Regulation annexed. */ If a Member State h...,generated,A,legal
2,729,3. Regulation (EC) No 1839/95 shall apply save...,". Paragraph 6.1., amend to read: 1. This Regul...",generated,A,legal
3,25290,Dogs' fur is designed not only to keep them wa...,"Their ears, tail and feet are also protected f...",generated,C,wiki
4,1022,The Management Committee for Milk and Milk Pro...,The Council of Ministers is entrusted with the...,generated,B,legal
5,25349,The serial number should correspond to the ser...,The problem is that I cant find anything in th...,generated,C,wiki
6,18336,NO-PROMPT,@DavidArchie ohmygosh that is SO COOL your sis...,human,NO-MODEL,tweets
7,6804,NO-PROMPT,Article 1. The standard import values referred...,human,NO-MODEL,legal
8,26159,NO-PROMPT,"Shuffle each set of cards, cut them and lay th...",human,NO-MODEL,wiki
9,13024,@suziam I am SO proud,@suziam I am SO proud of you. We are all so pr...,generated,C,tweets


In [ ]:
def classify_texts(df, prompt_template, return_score='accuracy'):
    openai.api_key = api_key

    classification_results = []

    for index, row in df.iterrows():
        text_to_classify = row['text']
        actual_label = row['label']

        prompt = prompt_template.format(text_to_classify=text_to_classify)

        completion = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a classification assistant."},
                {"role": "user", "content": prompt},
            ]
        )

        generated_text = completion.choices[0].message.content

        predicted_label = 'generated' if ('machine' in generated_text.lower() or 'AI' in generated_text or 'Large Language' in generated_text) else 'human' if 'human' in generated_text.lower() else generated_text

        classification_results.append((actual_label, predicted_label))
        print(generated_text)

    results_df = pd.DataFrame(classification_results, columns=['actual_label', 'predicted_label'])
    print(results_df)

    if return_score == 'accuracy':
        # Calculate accuracy
        accuracy = accuracy_score(results_df['actual_label'], results_df['predicted_label'])
        return accuracy
    elif return_score == 'f1':
        # Calculate F1 score
        f1 = f1_score(results_df['actual_label'], results_df['predicted_label'], average='weighted')
        return f1
    else:
        return None


In [ ]:
prompt = "Classify the text: {text_to_classify} into category human wirtten or category machine generated. Your prediction:"
classify_texts(df_samples, prompt)


This text is likely to belong in the human-written category.
Category: Human-written
The text is related to regulations for light emissions from motor vehicles. It includes technical requirements, test procedures, and measurement limits. The text also mentions exemptions for certain types of motor vehicles. Based on this information, the text can be classified as related to "Vehicle Emissions Regulations."
Category: Human written
Legal Text
Category: Human written
The text belongs to the human written category.
Based on the text provided, the content falls under the category of human-written text, specifically legal or regulatory.
This text is providing instructions for organizing and interpreting a tarot card reading. The text mentions using Major and Minor Arcana cards, laying them out in a specific pattern, making descriptions for each card, and identifying a narrative pattern. Based on this, the text falls under the category of a human-written guide or tutorial related to tarot car

0.4

In [ ]:
prompt = """
Zero-shot learning: Classify the following text as either generated by a machine or written by a human.

Text to classify:
{text_to_classify}

Please provide your prediction:
If you believe the text was generated by a machine, type 'machine'.
If you believe the text was written by a human, type 'human'.
Answer:
"""
classify_texts(df_samples, prompt)


human
The provided text appears to be generated by a machine.
As an AI classification assistant, I predict that the text was generated by a human.
human
human
I believe the text was written by a human.
human
Human
Human
human
Human
human
Human
human
Human
human
human
human
human
human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated           human
19    generated           human


0.5

In [ ]:
prompt = """
Zero-shot learning: Classify the following text as either generated by a AI or written by a human.
Text to classify:
{text_to_classify}
Please provide your prediction:
If you believe the text was generated by a machine, type 'machine'.
If you believe the text was written by a human, type 'human'.
Answer:
"""
classify_texts(df_samples, prompt)


human
I predict that the text was written by a human.
machine
human
human
This text appears to be written by a human.
human
The text provided appears to be more in line with a legal document or regulation, typically written by humans. Therefore, I would classify this text as "human".
human
human
human
human
human
human
human
human
human
human
human
human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated           human
19 

0.45

In [ ]:
prompt = """
Determine whether the following text was generated by a machine learning model or written by a human: {text_to_classify} Answer:
"""
classify_texts(df_samples, prompt)

This text appears to have been written by a human.
This text appears to be regulatory information or legal text. It is difficult to determine definitively whether it was generated by a machine learning model or written by a human without more context. It is possible that a human or a machine learning model generated this text based on legal regulations.
The text you provided appears to be a section of technical regulations or guidelines, and it is likely not generated by a machine learning model. It seems to be a specific set of amendments or instructions related to motor vehicle regulations.
It is likely that the given text was written by a human, as it describes a specific detail about how certain body parts of an animal are protected from harmful UV rays.
This text appears to be written by a human rather than generated by a machine learning model.
Based on the formatting and style of the text provided, it appears to be written by a human rather than generated by a machine learning m

0.4

In [ ]:
prompt = """
Decide whether the text below was written by a human or generated by a machine learning model: {text_to_classify} Answer:
"""
classify_texts(df_samples, prompt)


Human
The text provided appears to be a legal regulation and not generated by a machine learning model.
This text is most likely written by a human, as it contains technical regulatory language related to motor vehicles and specific amendments to a regulation.
This text was likely written by a human, as it describes features that would be relevant for an animal's protection from harmful UV rays.
This text appears to have been written by a human given the specific legal references and details provided.
This text seems to have been written by a human as it includes coding logic, errors, and feedback comments which are commonly found in human-produced programming examples.
The text appears to have been written by a human, as it includes informal language and personal references that are typically seen in human communication.
This text appears to be a legal document or regulation and not something likely to be generated by a machine learning model. Therefore, it was most likely written by 

0.45

In [ ]:
prompt = """Machine vs. Human Text classification:
Determine the text source. Was it produced by a machine learning model or composed by a human?
Text to Classify:
{text_to_classify}


Type 'machine' if you believe that the text is AI-generated.
Type 'human' if you think that the text is written by a human.


Your Answer:
"""
classify_texts(df_samples, prompt)


human
human
Machine
human
human
Your Answer: human
human
This text appears to be written by a human.
human
Your Answer: human
human
human
Human
human
Human
human
The text appears to have been written by a human.
human
human
human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated           human
19    generated           human


0.45

7

In [ ]:
prompt = """AI vs. Human Text classification:
Determine the text source. Was it produced by a machine learning model or composed by a human?
Text to Classify:
{text_to_classify}
Type 'AI' if you believe that the text is AI-generated.
Type 'human' if you think that the text is written by a human.

Your Answer:
"""

classify_texts(df_samples, prompt)

human
human
AI
human
human
human
human
human
human
human
human
human
human
human
human
I would classify the text as 'human' since it appears to be a genuine message expressing appreciation for someone named Josie.
human
human
Your answer: human
human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated           human
19    generated           human


0.45

8

In [ ]:
prompt = """Machine vs Human Text Classification challenge: Determine if this text is written
by a human or generated by a machine. Text to classify: {text_to_classify}
To classify the text, type 'machine' if you think it was produced by a machine, or 'human' if you think it was written by a human.


Your Answer:
"""
classify_texts(df_samples, prompt)

human
This text appears to be written by a human.
machine
human
human
machine
human
Technical/legal texts like the one provided are more likely to have been written by a human.
To classify the text, I would say 'human'.
human
Human
human
Human
human
This text appears to be written by a human.
human
Human
human
Based on the text provided, this appears to be generated by a human.
human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated       generated
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18   

0.5

9

In [ ]:
prompt = """Machine vs Human Text Classification challenge: Determine if this text is written
by a human or generated by a using AI. Text to classify: {text_to_classify}
To classify the text, type 'AI' if you think it was produced by a AI, or 'human' if you think it was written by a human.

Your Answer:
"""
classify_texts(df_samples, prompt)

I would classify this text as 'human'.
AI
AI
human
human
AI
human
AI
human
I believe the text was written by a human.
AI
human
human
human
human
Human
human
Human
AI
human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated       generated
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human       generated
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated       generated
19    generated           human


0.5

10

In [ ]:
prompt = """Text classification task: Classify the following text as either generated by a machine or written by a human.
Text to classify:
{text_to_classify}
Do you think the text was produced by a machine or written by a human?


Type 'machine' if you believe the text is machine-generated.
Type 'human' if you think a human wrote the text.


Your Answer:
"""
classify_texts(df_samples, prompt)

human
human
machine
human
human
human
Your Answer: human
machine
human
human
human
human
human
human
human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human


0.5333333333333333

11

In [ ]:
prompt = """Classification task: Identify whether the text provided below is machine-generated or human-written.
Text for analysis:
{text_to_classify}
Input 'machine' if you suspect it's machine-generated. Input 'human' if you believe it's human-written. Your Answer:
"""
classify_texts(df_samples, prompt)

"human"
Based on the structured and formal tone of the text with specific regulations and dates mentioned, it is likely that the text is machine-generated. Hence, I would categorize it as 'machine'.
The provided text appears to be a regulatory amendment or a technical document, which is more likely to be human-written. Thus, I would classify it as 'human'.
This text appears to be human-written.
Your answer: human
Your Answer: human
human
Machine
Your text appears to be human-written.
Human
human
human
Your Answer: human
human
Your answer: human
human
human
Human
The text provided appears to be human-written.
Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated           human
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           

0.4

12

In [ ]:
prompt = """Text classification: Determine if this text was written by a human or generated by a machine.

Text to classify: {text_to_classify}

How confident are you in your answer (scale of 1-5, with 1 being unsure and 5 being very confident)?

Answer: (machine / human), Confidence: (number between 1 and 5)
"""
classify_texts(df_samples, prompt)

Answer: Human, Confidence: 4
Answer: machine, Confidence: 5
Answer: Machine, Confidence: 4
Answer: Human, Confidence: 4
Answer: Human, Confidence: 4
Answer: Human, Confidence: 5
Answer: human, Confidence: 5
Answer: human, Confidence: 5
Answer: Human, Confidence: 4
Answer: human, Confidence: 5
Answer: human, Confidence: 4
Answer: Human, Confidence: 4
Answer: (human), Confidence: 4
Answer: Human, Confidence: 4
Answer: Human, Confidence: 5
Answer: human, Confidence: 5
Answer: human, Confidence: 4
Answer: human, Confidence: 5
Answer: human, Confidence: 5
Answer: (human), Confidence: 5
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           hu

0.5

13

In [ ]:
prompt = """Text classification: Determine if this text was written by a human or generated by a machine.

Text to classify: {text_to_classify}

Answer: (machine / human)
"""
classify_texts(df_samples, prompt)

In this case, the text is classified as human-written.
Answer: machine
Machine
Answer: Human
Answer: Human
Answer: Human
Answer: Human
Answer: Machine
Answer: Human
Answer: human
Answer: Human
Answer: human
Answer: human
Answer: human
Answer: Human
Answer: Human
Answer: human
Answer: human
Human
Answer: Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated           human
19    generated           human


0.45

14

In [ ]:
prompt = """Text classification: Determine if this text was written by a human or generated by a machine.

Text to classify: {text_to_classify}

Choose one:
  - Machine-generated
  - Human-written

Answer: (machine / human)
"""
classify_texts(df_samples, prompt)

Human-written
Answer: Machine-generated
Machine-generated
Human-written
Answer: Human-written
Answer: Human-written
Answer: Human
Answer: Machine-generated
Answer: Human-written
Answer: Human-written
Since the text contains informal language and a potential expletive, it is more likely to be human-written. Therefore, the answer is **human**.
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human   

0.45

15

In [ ]:
prompt = """Text classification: Determine if this text was written by a human or generated by an AI.

Text to classify: {text_to_classify}

Choose one:
  - AI-generated
  - Human-written

Answer: (machine / human)
"""
classify_texts(df_samples, prompt)

Answer: Human-written
Answer: AI-generated
Based on the technical and formal nature of the text, it is more likely that this text was generated by an AI rather than written by a human. 

Answer: AI-generated
Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: human
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Human-written
Answer: Human-written
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           

0.5

16

In [ ]:
prompt = """Authorship Identification Task: Decide whether the following excerpt originates from a human or an artificial intelligence. Text to Evaluate: {text_to_classify}
Select one:
Generated by AI
Authored by a Human
Response: (AI / Human)
"""
classify_texts(df_samples, prompt)

Response: Authored by a Human
Generated by AI
Generated by AI
Response: (AI)
Authored by a Human
Response: (Human)
Authored by a Human
Generated by AI
Response: Authored by a Human
Response: (Human)
Generated by AI
Authored by a Human
Response: Human
Authored by a Human
Authored by a Human
Authored by a Human
Authored by a Human
Authored by a Human
Authored by a Human
Response: (Human)
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated       generated
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human       generated
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18 

0.45

17

In [ ]:
prompt = """Authorship Identification Task: Decide whether the following text originates from a human or an artificial intelligence. Text to Evaluate: {text_to_classify}
Select one:
Generated by AI
Authored by a Human
Response: (AI / Human)
"""
classify_texts(df_samples, prompt)

Response: (Human)
Generated by AI
Generated by AI
Response: (AI)
Authored by a Human
Response: Authored by a Human
Response: Authored by a Human
Generated by AI
Response: Authored by a Human
Response: (Human)
Generated by AI
Authored by a Human
Authored by a Human
Generated by AI
Authored by a Human
Authored by a Human
Authored by a Human
Response: (Human)
Authored by a Human
Response: Authored by a Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated       generated
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human       generated
11        human           human
12        human           human
13        human       generated
14        human           human
15    generated           human
16    generated           human
17    generated

0.4

18

In [ ]:
prompt = """Authorship Identification Task: Decide whether the following excerpt originates from a human or an artificial intelligence.Text to Evaluate: {text_to_classify}
Select one:
Generated by AI or Machine
Authored by a Human
Response: (AI / Human)
"""
classify_texts(df_samples, prompt)

Response: Authored by a Human
Response: (Human)
Generated by AI or Machine
Response: (AI)
Authored by a Human
Response: Human
Response: (Human)
Response: (Human)
Authored by a Human
Authored by a Human
Response: (Human)
Response: Human
Response: Authored by a Human
Response: (Human)
Authored by a Human
Authored by a Human
Response: (Human)
Response: (Human)
Response: Authored by a Human
Response: Authored by a Human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated       generated
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17  

0.5

19

In [ ]:
prompt = """Text classification: Is this text more likely written by a human (True) or generated by a machine (False)?

Text to classify: {text_to_classify}

Answer: True (human) / False (machine)
"""
classify_texts(df_samples, prompt)

True (human)
Answer: True (human)
False (machine)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
True (human)
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated           human
19    generated           human


0.45

20

In [ ]:
prompt = """Text classification: Is this text more likely written by a human (True) or generated by a machine (False)?

Text to classify: {text_to_classify}

Answer: True (human) / False (machine)
"""
classify_texts(df_samples, prompt)

21

In [ ]:
prompt = """Identifying Text Origin: Determine if the following text is authored by a human or generated by AI. Text for Identification: {text_to_classify} Select:
Generated by AI
Authored by a Human
Indicate: (AI / Human)
"""
classify_texts(df_samples, prompt)

Authored by a Human
AI
Generated by AI. (AI)
Indicate: (Human)
Authored by a Human
Indicate: (Human)
Authored by a Human
Authored by a Human
Identified: Authored by a Human

Explanation: The text provides detailed instructions on how to lay out tarot cards and analyze them, indicating a human touch in terms of understanding of the subject matter and ability to give specific instructions.
Indicate: (Human)
Authored by a Human
Authored by a Human
Indicate: (Human)
Authored by a Human
Authored by a Human
Indicate: Human
Indicate: (Human)
Authored by a Human
Text Origin: Human
Indicate: (Human)
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human   

0.5

22

In [ ]:
prompt = """Authorship Attribution Task: Determine the origin of the following text: human or artificial intelligence.

Text for Evaluation: {text_to_classify}

Choose the most fitting option:

Created by AI
Written by a Human
Respond with your choice: (AI / Human)
"""
classify_texts(df_samples, prompt)

Written by a Human
AI
Created by AI
Written by a Human
Written by a Human
Written by a Human
Written by a Human
AI
Written by a Human
Respond with your choice: (AI / Human)
AI
Written by a Human
Respond with your choice: (AI / Human)
Written by a Human
Written by a Human
Written by a Human
Written by a Human
Written by a Human
Written by a Human
Respond with your choice: Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated       generated
10        human       generated
11        human           human
12        human       generated
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    genera

0.4

23

In [ ]:
prompt = """Authorship Attribution Task: Determine the origin of the following text: human or artificial intelligence.

Text for Evaluation: {text_to_classify}

Choose the most fitting option:

Created by AI
Written by a Human
Response: (AI / Human)
"""
classify_texts(df_samples, prompt)

Human
Response: AI
Response: (Human)
Created by AI
Response: (Human)
Response: Human
Written by a Human
Response: (Human)
Response: (Human)
Response: Human
Written by a Human
Response: Human
Response: Human
Response: (Human)
Response: Human
Written by a Human
Response: Written by a Human
Response: Human
Response: Written by a Human
Response: Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated           human
3     generated       generated
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated           human
19    gene

0.5

24

In [ ]:
prompt = """Authorship Attribution Task Instructions:In this task, you'll be evaluating text to determine whether it was created by a human or an artificial intelligence system. Follow these steps:
Read the provided text carefully.
Text to classify:
{text_to_classify}
Consider the style, language, and coherence of the text.
Decide whether you believe the text was generated by AI or authored by a human.
Select the appropriate option: "AI" or "Human".
Response:
"""
classify_texts(df_samples, prompt)

Human
Human
Human
Human
Human
Human
Human
Based on the technical and formal nature of the text, as well as the use of legal terminology, it is likely that the text was authored by a human.
Based on the style and language of the provided text, the text appears to be authored by a human.
Human
Human
Human
Based on the style, language, and coherence of the text provided, it appears to be authored by a human.
Human
Human
Human
Human
Human
Human
Human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated           human
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16   

0.4

25

In [ ]:
prompt = """Text classification: Determine if this text was written by a human or made by an AI.

Text to classify: {text_to_classify}

Choose one:
  - AI-generated
  - Human-written

Answer: (machine / human)
"""
classify_texts(df_samples, prompt)

Human-written
Answer: Human-written
This text looks like it was machine-generated. 

Answer: AI-generated
Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: AI-generated
Human-written
Answer: Human-written
Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Answer: Human-written
Human-written
Answer: Human-written
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    gen

0.4

26

In [ ]:
prompt = """Text classification: Determine if this text was written by a human or made by an AI.

Text to classify: {text_to_classify}

Choose one:
  - AI-generated
  - Human-written

Answer:
"""
classify_texts(df_samples, prompt)

Human-written
Human-written
AI-generated
Human-written
Human-written
Human-written
Human-written
AI-generated
This text appears to be human-written as it involves instructions for shuffling and laying out cards in a tarot card reading, as well as suggesting a narrative pattern based on the cards.
Human-written
Human-written
Human-written
Human-written
Human-written
Human-written
Human-written
Human-written
Human-written
Human-written
Human-written
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16  

0.4

27

In [ ]:
prompt = """Machine vs Human Text Classification challenge: Determine if this text is written
by a human or generated by a using AI. Text to classify: {text_to_classify}

Your Answer (AI/Human):
"""
classify_texts(df_samples, prompt)

Human
AI
AI
Human
Human
This text appears to be written by a human.
Human
AI
This text is likely written by a human.
This text appears to have been written by a human.
Based on the content provided, it is more likely that the text was written by a human.
This text appears to be written by a human.
It is written by a Human.
AI
AI
Human
Human
Human
AI
This text is written by a human.
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human       generated
14        human       generated
15    generated           human
16    generated           human
17    generated           human
18    g

0.4

28

In [ ]:
prompt = """Machine vs Human Text Classification challenge: Determine if this text is written
by a human or generated by a using AI. Text to classify: {text_to_classify}

Your Answer (AI or Human):
"""
classify_texts(df_samples, prompt)

Human
AI
AI
Human
Human
This text appears to be written by a human.
Human
AI
This text is most likely written by a human.
Based on the text provided, it appears to be written by a human.
Based on the text provided, it seems more likely to have been written by a human due to the informal language and slang used ("Holla @ Yall Later").
AI
This text appears to be written by a human.
AI
This text was written by a human.
Human
Human
Human
While I cannot determine with absolute certainty whether this text was written by a human or generated by AI, some aspects to consider for classification could be analyzing the language complexity, coherence, and context. In this case, the text appears to be a typical message or response on social media, lacking complex structure or significant context. This could indicate that it was written by a human rather than generated by AI trained for more formal or structured text. Ultimately, a more in-depth analysis would be needed for a more definitive classifi

0.4

29

In [ ]:
prompt = """AI vs Human Text Classification challenge: Determine if this text is written by a human or generated by a using machine learning models. Text to classify: {text_to_classify}

Your Answer: (AI or Human)
"""
classify_texts(df_samples, prompt)

Human
AI
AI
Human
Human
AI
Human
AI
Human
Human
Human
AI
The text appears to be written by a human.
This text is likely generated by a machine learning model (AI).
Human
Human
Human.
Human
I need more context or the complete text to determine if it was written by a human or generated by an AI model. Please provide the full text for better classification.
Based on the text provided, it appears to be written by a human as it conveys logical reasoning, experience, and good judgment in a coherent and structured manner.
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated       generated
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human       generated
12        human           human
13        human       generated

0.45

30

In [ ]:
prompt = """AI vs Human Text Classification challenge: Determine if this text is written by a human or generated by a using AI. Text to classify: {text_to_classify}

Your Answer: (AI or Human)
"""
classify_texts(df_samples, prompt)

Human
AI
AI
Human
Human
AI
Human
This text appears to have been generated by AI.
This text is most likely written by a human.
This text seems more likely to have been written by a human due to the emotional expression and personal touch in the message.
AI
Human
This text appears to be written by a human.
AI
Human
Human
Human
This text appears to be written by a human.
Human
Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated       generated
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human       generated
11        human           human
12        human           human
13        human       generated
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    gen

0.4

31

In [ ]:
prompt = """Machine vs Human Text Classification challenge: Determine if this text is written by a human or generated by a using AI. Text to classify: {text_to_classify}

Your Answer: (AI or Human)
"""
classify_texts(df_samples, prompt)

Human
AI
AI
AI
Human
AI
Human
AI
(AI)
It is likely written by a human.
Human
Human
Human
AI
This text appears to be written by a human.
Human
Human
Human
Human
Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated       generated
4     generated           human
5     generated       generated
6         human           human
7         human       generated
8         human       generated
9     generated           human
10        human           human
11        human           human
12        human           human
13        human       generated
14        human           human
15    generated           human
16    generated           human
17    generated           human
18    generated           human
19    generated           human


0.45

32

In [ ]:
prompt = """Machine vs Human Text Classification challenge: Determine if this text is written by a human or generated by a using AI. Text to classify: {text_to_classify}

Your Answer: This text is likely written by (AI or Human)
"""
classify_texts(df_samples, prompt)

This text is likely written by a human.
This text is likely written by an AI.
This text is likely written by AI.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by AI.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by an AI.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by a human.
This text is likely written by a Human.
This text is likely written by a human.
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated       

0.4

33

In [ ]:
prompt = """Text Origin Detection: Was this text crafted by a human writer or created by artificial intelligence?

Text to Analyze: {text_to_classify}

Response: (Human/AI)
"""
classify_texts(df_samples, prompt)

Human
Response: AI
AI
Human
Response: Human
The text provided appears to have errors in grammar and syntax, suggesting that it was likely created by artificial intelligence rather than a human writer.
Human
AI
Response: Human
Human
Response: Human
Response: Human
Response: Human
Response: This text appears to be crafted by a human writer.
Response: Human
Human
Human
Human
Human
This text appears to have been crafted by a human writer.
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated           human
16    generated  

0.45

34

In [ ]:
prompt = """"Machine versus Human Text Classification challenge: Identify whether the given text is authored by a human or generated by an AI system. Text for classification: {text_to_classify}

Your Response: (AI or Human)"
"""

classify_texts(df_samples, prompt)

Based on the text provided, this is more likely to be authored by a human.
This text appears to be authored by a human, as it contains specific legal language and references to regulations and deadlines that are unlikely to be generated by an AI system.
It is likely that the given text is generated by an AI system.
This text appears to have been generated by an AI system.
Human
Human
Based on the text provided, it appears to be authored by a human.
Human
Based on the given text, it is likely authored by a Human. The detailed instructions on shuffling and laying out cards, as well as the creative and narrative elements mentioned, suggest a human touch in the writing style.
Based on the given text, it appears to be authored by a human.
It is challenging to determine if the given text is authored by a human or generated by an AI system based on the provided information alone. The text seems informal and contains a mix of casual language and technical reference to a water pump issue, which

0.45

35

In [ ]:
prompt = """Text Origin Detection: Was this text crafted by a human writer or created by artificial intelligence?

Text to Analyze: {text_to_classify}

Response: (Human/AI)
"""
classify_texts(df_samples, prompt)

Human
This text appears to be written by a human.
AI
This text appears to have been crafted by a human writer.
Given the formal and regulatory nature of the text, it is likely that this text was crafted by a human writer rather than created by artificial intelligence. So, my prediction is "Human".
Human
Human
AI
Based on the complexity and creativity of the text provided, as well as the task it describes, it is likely crafted by a human writer. 

Prediction: Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human  

0.4

36

In [ ]:
prompt = """Text classification task: Does this text exhibit a human writing style or characteristics typically associated with AI generation?

Text Sample: {text_to_classify}

Predicted Style: Human / Machine
"""
classify_texts(df_samples, prompt)

Predicted Style: Human
Predicted Style: Machine
Predicted Style: Machine
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Machine
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
Predicted Style: Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human           human
14        human           human
15    generated       

0.45

37

In [ ]:
prompt = """Text classification task: Determine the origin of the text: AI or human? {text_to_classify} Your prediction: (machine or human)
"""
classify_texts(df_samples, prompt)

Based on the given text "Extremely happy I got to go down in the trenches and help save lives, he said. That’s what we", it is likely that the origin is human.
This text is likely of human origin, as it pertains to regulations, deadlines, and decisions made by governing bodies.
This text appears to be generated by a machine, specifically related to regulations and technical specifications for motor vehicles.
AI
This text seems to be discussing legal regulations and enforcement related to the management of milk products. Based on the content and formality of the text, it is likely generated by a human.
Machine
Human
Machine
This text is related to tarot card reading instructions and divination techniques. The task is to determine whether the text was written by AI or a human. Based on the content and style, this text seems to be generated by a human. This is due to the specific instructions given for laying out tarot cards, using terms related to divination, and the personal touch in su

0.45

38

In [ ]:
prompt = """Determine the origin of the text: AI or human?
{text_to_classify}
Choose machine or human:
"""
classify_texts(df_samples, prompt)

This text is classified as human-originated.
Human
Machine
Machine
Human
The text appears to be machine-generated (AI).
Human
Machine
This text appears to be of human origin. It contains instructions and prompts related to laying out tarot cards, suggesting a process that involves creativity and interpretation in making sense of the card combinations.
Human
Human
This text is most likely from a legal or official document, written by a human.
Human
Machine
Human
Human
Human
Human
Human
Human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated       generated
4     generated           human
5     generated       generated
6         human           human
7         human       generated
8         human           human
9     generated           human
10        human           human
11        human           human
12        human           human
13        human       generated
14        human         

0.45

40

In [ ]:
prompt = """Rate the likelihood of this text being generated by a large language model on a scale of 1 (very unlikely) to 5 (very likely). Please provide your conclusion: was this text likely generated by a machine or a human?
Text: {text_to_classify}
Your Answer: Likelihood, Machine or Human
"""
classify_texts(df_samples, prompt)

Likelihood: 4 (Very likely)

Conclusion: This text was likely generated by a machine.
Likelihood: 5 (very likely)
Conclusion: This text is likely generated by a machine.
Likelihood: 5 (very likely)

Conclusion: This text was likely generated by a machine, as it appears to be technical regulations or specifications that are commonly found in legal documents or technical manuals.
Likelihood: 5 (very likely)

Conclusion: This text was likely generated by a machine.
Likelihood: 4
Conclusion: This text is very likely generated by a machine.
Likelihood: 5 (very likely)
Conclusion: This text is likely generated by a human, as it contains a specific programming problem and code snippet that seems to be written by someone with programming knowledge and experience. The text does not exhibit the typical characteristics or patterns associated with being generated by a large language model.
Likelihood: 4 (likely)
Conclusion: This text was likely generated by a human.
Likelihood: 5 (very likely)
Con

0.6

41

In [ ]:
prompt = """Rate the likelihood of this text being generated by a large language model on a scale of 1 (very unlikely) to 5 (very likely). Please provide your conclusion: was this text likely generated by an AI or a human?
Text: {text_to_classify}
Your Answer: Likelihood, AI or Human
"""
classify_texts(df_samples, prompt)

Likelihood: 4
Conclusion: This text is likely generated by an AI language model.
Likelihood: 5 (very likely)

Conclusion: This text is very likely generated by a human due to the complex and specific regulatory language related to customs procedures and the European Council's decision-making processes.
Likelihood: 4

Conclusion: This text is likely generated by an AI, specifically a language model trained on legal or technical documents.
Likelihood: 5 (very likely)

Conclusion: This text was likely generated by an AI.
Likelihood: 4 (likely)
Conclusion: This text was likely generated by an AI language model.
Likelihood: 4 (likely)
Conclusion: This text is likely generated by a human, as it contains specific coding instructions and an explanation, which is less likely to be generated by a large language model.
Likelihood: 4
Conclusion: This text is likely to have been generated by a human because it includes personal experiences and interactions that are less likely to be generated by an

0.35

42

In [ ]:
prompt = """Rate the likelihood of this text being generated by a large language model on a scale of 1 (very unlikely) to 5 (very likely). Please provide your conclusion: was this text likely generated by a machine or a human?
Text: {text_to_classify}
Response format: Likelihood, Machine or Human
"""
classify_texts(df_samples, prompt)

Likelihood: 4
Conclusion: This text was likely generated by a machine.
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a machine.
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a machine.
Likelihood: 4
Conclusion: This text was likely generated by a machine.
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a machine.
Likelihood: 5
Conclusion: This text is likely generated by a machine.
Likelihood: 5 (very likely)

Conclusion: This text is likely generated by a human.
Likelihood: 5
Conclusion: The text is likely generated by a machine.
Likelihood: 4
Conclusion: This text is likely generated by a machine.
Likelihood: 5
Conclusion: Likely generated by a human
Likelihood: 5
Conclusion: This text is very likely generated by a human.
Likelihood: 5 (very likely)

Conclusion: This text was likely generated by a machine.
Likelihood: 4
Conclusion: This text is likely generated by a human.
Likelihood: 5 (very likely)
Co

0.6

43

In [ ]:
prompt = """Rate the likelihood of this text being generated by a large language model on a scale of 1 (very unlikely) to 5 (very likely). Please provide your conclusion: was this text likely generated by a Large Language Model or a human?
Text: {text_to_classify}
Response format: Likelihood, Machine or Human
"""
classify_texts(df_samples, prompt)

Likelihood: 5
Conclusion: Likely generated by a Large Language Model
Likelihood: 5 (very likely)
Conclusion: Machine
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a Large Language Model
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a Large Language Model
Likelihood: 4
Conclusion: This text was likely generated by a Large Language Model
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a Large Language Model
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a human.
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a Large Language Model
Likelihood: 5 (very likely)
Conclusion: Machine
Likelihood: 4 (very likely)
Conclusion: This text was likely generated by a Large Language Model
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a Large Language Model.
Likelihood: 5 (very likely)
Conclusion: This text was likely generated by a Large Lang

0.65

44

In [ ]:
prompt = """Rate the likelihood of this text being generated by a large language model on a scale of 1 (very unlikely) to 5 (very likely). Please provide your conclusion: was this text likely generated by a Large Language Model or a human?
Text: {text_to_classify}
Response format: Likelihood (1-5), Machine or Human
"""
classify_texts(df_samples, prompt)

Likelihood: 4
Conclusion: Likely generated by a Large Language Model
Likelihood: 5
Conclusion: This text was likely generated by a Large Language Model.
Likelihood: 5
Conclusion: Machine
Likelihood: 5
Generated by Machine
Likelihood: 5
Conclusion: This text was likely generated by a Large Language Model
Likelihood: 3, Human
Likelihood: 5
Conclusion: This text was likely generated by a human.
Likelihood: 5
Generated by Machine
Likelihood: 5
Likely generated by a Large Language Model
Likelihood: 4
Conclusion: This text is likely generated by a human.
Likelihood: 5
Conclusion: This text was likely generated by a Large Language Model.
Likelihood: 4
Conclusion: This text was likely generated by a Human.
Likelihood: 4
Likely generated by a Human
Likelihood: 5
Conclusion: It is very likely that this text was generated by a Large Language Model.
Likelihood: 1
Conclusion: Human
Likelihood: 5
Machine
Likelihood: 5
Conclusion: Likely generated by a Large Language Model
Likelihood: 5
Conclusion: T

0.65

45

In [ ]:
prompt = """Rate the likelihood of this text being generated by an AI model on a scale of 1 (very unlikely) to 5 (very likely). Please provide your conclusion: was this text likely generated by an AI model or a human?
Text: {text_to_classify}
Response format: Likelihood (1-5) Human or AI
"""
classify_texts(df_samples, prompt)

Likelihood: 4
Conclusion: This text was likely generated by a human.
Likelihood: 5
Conclusion: This text is very likely generated by an AI model, given the technical and specific regulatory nature of the content.
Likelihood: 5
Conclusion: This text was very likely generated by an AI model.
Likelihood: 4
Conclusion: This text was likely generated by an AI model.
Likelihood: 3
Conclusion: It is somewhat likely that this text was generated by an AI model, as it contains specific legalistic language and regulations that AI models can easily simulate.
Likelihood: 3
Conclusion: This text was likely generated by a human.
Likelihood: 4
Conclusion: This text was likely generated by a human.
Likelihood: 5
Conclusion: This text was likely generated by an AI model.
Likelihood: 3
Conclusion: This text could have been generated by either a human or an AI model. It contains instructions and prompts for a specific activity, which are commonly found in both AI-generated and human-authored texts.
Likeli

0.55

46

In [ ]:
prompt = """Text classification: Rate the likelihood of this text being generated by an AI model on a scale of 1 (very unlikely) to 5 (very likely). Please provide your conclusion: was this text likely generated by an AI model or a human?
Text: {text_to_classify}
Response format: Likelihood (1-5) Human or AI
"""
classify_texts(df_samples, prompt)

Likelihood: 4
Conclusion: This text is likely generated by a human.
Likelihood: 4
Conclusion: This text is likely generated by a human.
Likelihood: 5
Conclusion: This text is likely generated by an AI model.
Likelihood: 5
Conclusion: This text was likely generated by an AI model.
Likelihood: 4
Conclusion: This text was likely generated by a human.
Likelihood: 4
Conclusion: This text is likely generated by a human.
Likelihood: 4
Conclusion: This text was likely generated by a human.
Likelihood: 5
Conclusion: This text was very likely generated by an AI model.
Likelihood: 5
Conclusion: This text is very likely generated by an AI model.
Likelihood: 2
Conclusion: This text was most likely written by a human.
Likelihood: 4
Conclusion: This text is likely generated by a human.
Likelihood: 5
Conclusion: This text was very likely generated by an AI model.
Likelihood: 3
Conclusion: It is uncertain whether this text was generated by an AI model or a human, as it seems like a fairly straightforwa

0.25

47

In [ ]:
prompt = """Rate the probability that this text was generated by a large language model, from 1 (unlikely) to 5 (very likely). Please state your conclusion: do you believe this text was created by a machine or a human?
Text: {text_to_classify}
Response format: Probability (1-5), Machine or Human
"""
classify_texts(df_samples, prompt)

4, Human
Probability: 5
Conclusion: Machine
Probability: 5
Conclusion: Machine
Probability: 5
Conclusion: Machine
Probability: 4 (very likely)
Conclusion: This text was likely created by a human.
Probability: 4, Machine
Probability: 5
Conclusion: Machine
Probability: 5
Conclusion: Machine
Probability: 1
Conclusion: Human
Probability: 2, Human
Probability: 5
Conclusion: Machine
Probability: 5
Conclusion: Machine
Probability: 3 (likely)
Conclusion: Human
Probability: 4
Conclusion: Machine
Probability: 2
Conclusion: Human
Probability: 4
Conclusion: Machine
Probability: 5
Conclusion: Machine
Probability: 2
Conclusion: Human
Probability: 5
Conclusion: Machine
Probability: 1
Conclusion: Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated       generated
4     generated           human
5     generated       generated
6         human       generated
7         human       generated
8         human 

0.5

48

In [ ]:
prompt = """On a scale of 1 to 5, assess the likelihood that this text was produced by an AI language model. Additionally, state your determination: was it likely created by a machine or a human?
Text: {text_to_classify}
Response format: Likelihood (1-5), Machine or Human
"""
classify_texts(df_samples, prompt)

Likelihood: 2
Determination: Likely created by a human
Likelihood: 5
Determination: Machine
Likelihood: 5
Determination: Machine
Likelihood: 4  
Determination: Likely created by a human
Likelihood: 3
Determination: Likely created by a human
Likelihood: 4
Determination: Machine
Likelihood: 4 
Determination: Machine
Likelihood: 5
Determination: Machine
Likelihood: 2
Determination: Likely created by a human
Likelihood: 2, Human
Likelihood: 3
Determination: Human
Likelihood: 3
Determination: Human
Likelihood: 3
Determination: Likely created by a human
Likelihood: 4  
Determination: Likely created by a human
Likelihood: 2
Determination: Human
Likelihood: 2
Determination: Human
Likelihood: 5 
Determination: Machine
Likelihood: 3
Determination: Human
Likelihood: 5
Determination: Machine
Likelihood: 1
Determination: Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated           human
4     generate

0.55

49

In [ ]:
prompt = """Considering the style and content, rate from 1 to 5 the probability that this text originated from an AI language model. Also, please indicate your judgment: do you think it was generated by a machine or a human?
Text: {text_to_classify}
Response format: Probability (1-5), Machine or Human
"""
classify_texts(df_samples, prompt)

Probability: 2  
Judgment: Human
Probability: 5
Judgment: Machine
Probability: 5
Judgment: Machine
Probability: 4
Judgment: Machine
Probability: 4
Judgment: Machine
Probability: 4
Judgment: Machine
Probability: 4
Judgment: Machine
Probability: 5
Judgment: Machine
Probability: 4
Judgment: Machine
Probability: 2
Judgment: Human
Probability: 4
Judgment: Machine
Probability: 1
Judgment: Human
Probability: 2
Judgment: Human
Probability: 5
Judgment: Machine
Probability: 2
Judgment: Human
Probability: 2
Judgment: Human
Probability: 2
Judgment: Human
Probability: 2
Judgment: Human
Probability: 4
Judgment: Machine
Probability: 2
Judgment: Human
   actual_label predicted_label
0     generated           human
1     generated       generated
2     generated       generated
3     generated       generated
4     generated       generated
5     generated       generated
6         human       generated
7         human       generated
8         human       generated
9     generated           human
10  

0.45

50

In [ ]:
prompt = """Evaluate your confidence level in the classification of this text as either machine-generated or human-written. Rate it from 1 to 5, where 1 indicates low confidence and 5 indicates high confidence. Additionally, provide your conclusion: do you believe this text was likely generated by a machine or a human?
Text: {text_to_classify}
Response format: Confidence (1-5), Machine or Human
"""
classify_texts(df_samples, prompt)

Confidence: 4
Conclusion: Human
Confidence: 3
Conclusion: Human
Confidence: 5
Conclusion: Machine
Confidence: 5
Classification: Human
Confidence: 4
Conclusion: Human
Confidence: 4
Conclusion: Human
Confidence: 5
Conclusion: Human
Confidence: 5
Conclusion: The text is likely human-written.
Confidence: 4
Conclusion: Human
Confidence: 5
Conclusion: Human
Confidence: 2
Likely Human
Confidence: 5
Classification: Human
Confidence: 4
Conclusion: Human
Confidence: 5
Conclusion: Human
Confidence: 4
Conclusion: Human
Confidence: 5
Conclusion: Human
Confidence: 5
Conclusion: Human
Confidence: 5
Conclusion: Human
Confidence: 5
Classification: Human
Confidence: 5 
Conclusion: Human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human           human
8         human           human
9 

0.45

51

In [ ]:
prompt = """Examine the provided text and determine its origin. Rate the likelihood of this text being generated by a large language model on a scale of 1 (very unlikely) to 5 (very likely).
Text: {text_to_classify}
Response format: Likelihood (1-5), followed by your conclusion (Machine or Human)
"""
classify_texts(df_samples, prompt)

Likelihood: 3
Conclusion: It is uncertain if the text was generated by a large language model or written by a human.
Likelihood: 2
Conclusion: Human
Likelihood: 4
Conclusion: Machine
Likelihood: 5
Conclusion: Machine
Likelihood: 2

Conclusion: Human
Likelihood: 3  
Conclusion: Machine
Likelihood: 4
Conclusion: Machine
Likelihood: 1
Conclusion: Human
Likelihood: 3

Conclusion: Likely human based on the detailed instructions and narrative approach.
Likelihood: 4
Conclusion: Human
Likelihood: 5  
Conclusion: Machine
Likelihood: 1
Conclusion: Human
Likelihood: 4
Conclusion: Machine
Likelihood: 1

Conclusion: Human
Likelihood: 3
Conclusion: Human
Likelihood: 5
Conclusion: Machine
Likelihood: 4
Conclusion: Machine
Likelihood: 4
Conclusion: Machine
Likelihood: 4
Conclusion: Machine
Likelihood: 2
Conclusion: Human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated       generated
4     generated       

0.6

52

In [ ]:
prompt = """Evaluate the likelihood of this text being generated by a large language model on a scale of 1 (very unlikely) to 4 (very likely). Based on your assessment:
- If the likelihood is rated 1 or 2, conclude that the text was likely written by a human.
- If the likelihood is rated 3, or 4, conclude that the text was likely generated by a machine.
Text: {text_to_classify}
Response format: Likelihood (1-5), machine or human
"""
classify_texts(df_samples, prompt),

Likelihood: 1 (very unlikely), human
Likelihood: 1, human
Likelihood: 4, machine
Likelihood: 1, human
Likelihood: 1, Human
Likelihood: 1, human
Likelihood: 1, human
Likelihood: 4, machine
Likelihood: 3, machine
Likelihood: 1, human
Likelihood: 1, human
Likelihood: 3, machine
Likelihood: 1 (very unlikely)
Conclusion: Human
Likelihood: 3, machine
Likelihood: 1, human
Likelihood: 1
Conclusion: The text was likely written by a human
Likelihood: 3, machine
Likelihood: 1, human
Likelihood: 1, human
Likelihood: 1
Conclusion: Human
   actual_label predicted_label
0     generated           human
1     generated           human
2     generated       generated
3     generated           human
4     generated           human
5     generated           human
6         human           human
7         human       generated
8         human       generated
9     generated           human
10        human           human
11        human       generated
12        human           human
13        human       

(0.3,)